# Lab 5 - Marginesy i kernele

**Wykonanie rozwiązań: Marcin Przewięźlikowski**

https://github.com/mprzewie/ml_basics_course

In [ ]:
import sys
sys.path.append("../lab2")

import numpy as np
import matplotlib.pyplot as plt
from dataset_ops import load_dataset
from typing import Tuple, List
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from time import time
%matplotlib inline

## Pomiary klasyfikacji
Wygenerujmy rozkład punktów podobny jak na powyższym obrazku. 

In [ ]:
X, y = load_dataset("ds_1.png", dropout=0.95)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_all = []
for x_0 in np.linspace(0, 100, 30):
    for x_1 in np.linspace(0, 100, 30):
        X_all.append([x_0, x_1])
X_all = np.array(X_all)

In [ ]:
for c in range(y.max() + 1):
    X_c = X_train[y_train==c]
    plt.scatter(X_c[:, 0], X_c[:, 1], c=("g" if c==0 else "k")) 
plt.title("Training dataset")
plt.show()

In [ ]:
for c in range(y.max() + 1):
    X_c = X_test[y_test==c]
    plt.scatter(X_c[:, 0], X_c[:, 1], c=("g" if c==0 else "k"))
plt.title("Test dataset")
plt.show()

Dla takiego rozkładu zbadajmy jak będą się zachowywały w zależności od wybranego współczynnika C (rozumianego jako - współczynnik sterujący równowagą między zwiększaniem marginesu, a zmniejszaniem ilości punktów po złej stronie granicy) następujące wartości (do przetestowania rozsądny zakres i ilość współczynników C - tak aby pokazać trend na wykresie):
* jaka jest szerokość marginesu;
* jaki % punktów znalazł się po "niewłaściwej" stronie płaszczyzny dzielącej klasy.

In [ ]:
Cs = 10 ** np.linspace(-5, 2, 15)
# Cs = np.array([1])
Cs_log = np.log10(Cs)
Cs

C jest współczynnikiem, razy który w trakcie treningu będzie mnożony koszt wynikający z niedokładności klasyfikacji. Koszt wynikający z szerokości marginesu decyzyjnego można chyba interpretować jako regularyzację L2.

In [ ]:
def svm_metrics(kernel: str, C: float, **kwargs) -> Tuple[float, float, float, float]:
    svm = SVC(kernel=kernel, C=C, **kwargs)
    t_start = time()
    svm.fit(X_train, y_train)
    t = time() - t_start
    acc_train = svm.score(X_train, y_train)
    acc_test = svm.score(X_test, y_test)    
    scores = svm.decision_function(X_all)
    scores_in_margin = np.abs(scores) < 1
    # szerokość marginesu jest proporcjonalna do stosunku 
    # liczby punktów, które się w nim znalazły do liczby wszystkich punktów
    # (coś na kształt metody monte carlo)
    margin_width = scores_in_margin.sum() / len(X_all)
    plt.show()
    return acc_train, acc_test, margin_width, t, svm

In [ ]:
def svm_metrics_measurements(kernel: str, **kwargs) -> Tuple[List[float], ...]:
    accuracies = []
    widths = []
    times = []
    svms = []
    for C in Cs:
        acc_train, acc_test, width, t, svm = svm_metrics(kernel, C, **kwargs)
        accuracies.append((acc_train, acc_test))
        widths.append(width)
        times.append(t)
        svms.append(svm)
    return accuracies, widths, times, svms

In [ ]:
def plot_svm_metrics_measurements(**kwargs):
    accuracies, widths, times, _ = svm_metrics_measurements(**kwargs)
    kwargs_str = ", ".join([f"{k} = {v}" for (k,v) in kwargs.items()])
    plt.figure(figsize=(6, 12))
    plt.subplot(3, 1, 1)

    plt.title(f"Metrics of SVM with {kwargs_str} \n\n Accuracy")
    plt.xlabel("$log_{10}$(C)")
    plt.ylabel("accuracy")
    plt.plot(Cs_log, [a[0] for a in accuracies], label="train")
    plt.plot(Cs_log, [a[1] for a in accuracies], label="test")
    plt.legend()
    plt.subplot(3, 1, 2)
    plt.title("Margin width (percentage of all points with abs(decision) < 1)")
    plt.xlabel("$log_{10}$(C)")
    plt.ylabel("margin width")
    plt.plot(Cs_log, widths)
    
    plt.subplot(3, 1, 3)
    plt.title("Fitting time")
    plt.xlabel("$log_{10}$(C)")
    plt.ylabel("time")
    plt.plot(Cs_log, times)
    plt.tight_layout()
    plt.show()

Dokonajmy tych samych obliczeń dla:

### Zwykłego SVM, 

In [ ]:
plot_svm_metrics_measurements(kernel="linear")

Widać, że ze wzrostem $C$ maleje szerokość marginesu, co oznacza że im większe $C$, tym bardziej zależy nam na dokładności klasyfikacji, a mniej na wyraźnej granicy decyzyjnej.

Ze wzrostem $C$ rośnie też czas treningu. Dokładność klasyfikacji zależy od większej liczby zmiennych (musimy dopasować się do przypadków treningowych) niż szerokości granicy decyzyjnej (dopasowujemy tylko jeden parametr). Ma zatem sens, że dopasowywanie dokładności klasyfikacji zajmie więcej iteracji.

### SVM z kernelem wielomianowym stopnia trzeciego

In [ ]:
plot_svm_metrics_measurements(kernel="poly",  gamma="scale", coef0=3)

Podobnie jak przy klasyfikacji, ze wzrostem $C$ maleje margines, ale także dokładność klasyfikacji zbioru testowego (podczas gdy rośnie dokładność na zbiorze treningowym). Mamy wtedy do czynienia z overfittingiem modelu.

### SVM z jakimś kernelem RBF (dla kilku wariantów współczynnika sterującego, jeżeli jest taki - miejmy pełną świadomość z jakiej RBF korzystamy!). 

In [ ]:
for gamma in 10 ** np.linspace(-5, 5, 5):
    plot_svm_metrics_measurements(kernel="rbf", gamma=gamma)

SVMy z kernelami RBF overfittują dla $C > 1$. Nie udaje im się też osiągnąć dużej pewności w predykcjach - prawie wszystkie punkty mają dokładność klasyfikacji w zakresie $(-1, 1)$.

Nie widać też dużej korelacji między $C$ a czasem treningu.

## Wizualizacja klasyfikacji
Na koniec zwizualizujmy też efekty działania poprzez odpowiednie pomalowanie płaszczyzny (tak jak robiliśmy to przy metodzie k-NN). Z jednym wyjątkiem - tym razem niech odcień danego piksela zależy od odległości od płaszczyzny podziału (bliskie punkty = sporne = jasne, dalekie punkty = jednoznaczne = ciemne).

Opis kolorów:

* czarny - punkty treningowe klasy $1$
* zielony - punkty treningowe klasy $2$
* czerwony - obszar zaklasyfikowany jako klasa $1$
* żółty - obszar zaklasyfikowany jako klasa $2$


In [ ]:
def visualize_svm(svm:SVC):
    svm.fit(X_train, y_train)
    decisions = svm.decision_function(X_all)
    for x_s, d in zip(X_all, decisions):
        a = min(np.abs(d), 1)
        plt.scatter([x_s[0]], [x_s[1]], c=["r" if d > 0 else "y"], alpha=a)
    plt.scatter(X_train[:, 0], X_train[:, 1], 
                c=[("g" if y==0 else "k") for y in y_train]
               )
    plt.title(f"classification of {svm}")
    plt.show()

### SVM z kernelami liniowymi

In [ ]:
for svm in [
    SVC(kernel="linear", C=10e-4),
    SVC(kernel="linear", C=1),
    SVC(kernel="linear", C=100),

]:
    visualize_svm(svm)

Dla danego datasetu SVMy liniowe mają podobne granice decyzyjne, niezależnie od parametru C.

### SVM z kernelami wielomianowymi

In [ ]:
for svm in [
    SVC(kernel="poly", coef0=0, C=10e-5, gamma="scale"),
    SVC(kernel="poly", coef0=0, C=1, gamma="scale"),

    SVC(kernel="poly", coef0=2, C=10e-5, gamma="scale"),
    SVC(kernel="poly", coef0=2, C=1, gamma="scale"),

    SVC(kernel="poly", coef0=4, C=10e-5, gamma="scale"),
    SVC(kernel="poly", coef0=4, C=1, gamma="scale"),
    
    SVC(kernel="poly", coef0=8, C=10e-5, gamma="scale"),
    SVC(kernel="poly", coef0=8, C=1, gamma="scale"),
]:
    visualize_svm(svm)

Widać, że z małym $C$ (czyli dużą regularyzacją) SVM ma dość prostą granicę decyzyjną, niezależnie od stopnia wielomianu jaki dopasowuje. Zwiększając C, pozwalami modelowi nauczyć się granicy decyzyjnej bardziej biorącej pod uwagę punkty będące outlierami - a więc bardziej zoverfittować się do danych treningowych. Im większy stopień wielomianu, tym mniej "niepewnych" obszrów decyzyjnych.

### SVM z kernelami RBF

In [ ]:
for svm in [
    SVC(kernel="rbf", C=10e-5, gamma=10e-5),
    SVC(kernel="rbf", C=1, gamma=10e-5),
    SVC(kernel="rbf", C=100, gamma=10e-5),


    SVC(kernel="rbf", C=10e-5, gamma=10e-3),
    SVC(kernel="rbf", C=1, gamma=10e-3),
    SVC(kernel="rbf", C=100, gamma=10e-3),

    
    SVC(kernel="rbf", C=10e-5, gamma=10e-2),
    SVC(kernel="rbf", C=1, gamma=10e-2),
    SVC(kernel="rbf", C=100, gamma=10e-2),

]:
    visualize_svm(svm)

Dla takich samych wartości parametru $\gamma$, wzrost parametru $C$ pozwala modelowi bardziej dopasować się do danych treningowych i ustalić obszary obecności danej klasy w oparciu o obecność punktów należących do niej.

Im większy parametr $\gamma$, tym te obszary wydają się być węższe i tym bardziej rośnie obszar niepewny.